In [1]:
# load necessary components
from trafilatura import fetch_url, extract
import re
import os
from selenium import webdriver
from time import time

from selenium.webdriver import Keys, ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from openai import OpenAI


In [2]:
from trafilatura import fetch_url, extract

# The URL of the website to scrape
url = "https://www.wongnai.com/recipes" 

# Fetch the HTML content of the webpage
downloaded = fetch_url(url)

# Extract URLs from the fetched content
result = extract(downloaded, include_links=True)

print(result)

รวมเมนูอาหาร สูตรอาหารล่าสุด
เรียงตาม
สูตรล่าสุด
[เค้กโยเกิร์ต](recipes/yoghurt-cake)
อ่าน 20 ครั้ง
ชวนคนรักขนมหวานมาทำ “โยเกิร์ตเค้ก” เนื้อนุ่มฟูละมุนลิ้น ทำง่าย ๆ ใช้วัตถุดิบเพียงแค่ไม่กี่อย่าง ที่สำคัญมีไขมันต่ำ เป็นมิตรกับสุขภาพ ถูกใจสายเฮลตี!
ส่วนผสม: แป้งมันสำปะหลังต้นสน, โยเกิร์ต, น้ำตาล, แป้งเค้ก, ไข่ไก่, ผลไม้สด, น้ำตาลไอซิ่ง, วิปครีมสูตรอาหารโดย
กิน&GO
[ข้าวหน้าไก่ราชวงศ์](recipes/rice-with-chicken-gravy)
อ่าน 647 ครั้ง
ชวนมาทำ “ข้าวหน้าไก่ราชวงศ์” ข้าวหน้าไก่สุดคลาสสิกสไตล์ร้านดัง ข้าวนุ่มหอมเต็มเมล็ด ราดน้ำฉ่ำ ๆ หอมยั่วใจ ไก่เนื้อนุ่มเน้น ๆ เต็มคำ ทำยังไงตามมาดูกันเลย!
ส่วนผสม: เอโร่ ข้าวหอมปทุมธานี, ไก่, ซีอิ๊วขาว, ซีอิ๊วดำหวาน, พริกไทยป่น, แป้งข้าวโพด, แป้งข้าวโพด, น้ำมันงา, น้ำมันพืช, น้ำสต๊อก, ซอสปรุงรส, น้ำตาลทราย, พริกชี้ฟ้าเขียว, ผักชีสูตรอาหารโดย
กิน&GO
[เบอร์รีแฟลตครัวซองต์แซนด์วิช](recipes/berry-flat-croissant-sandwich)
อ่าน 455 ครั้ง
ปิดเทอมหน้าร้อนนี้ชวนเด็ก ๆ มาทำ “เบอร์รีแฟลตครัวซองต์แซนด์วิช” แป้งครัวซองต์แบนกรอบสุดฮิต สอดไส้ไอศกรีมและซอสมิกซ์เบอร์รี หวานฉ่ำก

In [3]:
import re

pattern = r'\(([^)]+)\)'

links = re.findall(pattern, result)
print(links)

['recipes/yoghurt-cake', 'recipes/rice-with-chicken-gravy', 'recipes/berry-flat-croissant-sandwich', 'recipes/chinese-sausage-baked-rice', 'recipes/jalapeno-bite', 'recipes/garlic-and-pepper-flavored-rice-noodles-with-shredded-chicken', 'อกไก่', 'recipes/thai-barbecue-gimbap', 'recipes/apple-cider-latte', 'recipes/japanese-salmon-fried-rice', 'recipes/flower-shaped-kanom-sai-sai', '/food-tips']


In [4]:
links

['recipes/yoghurt-cake',
 'recipes/rice-with-chicken-gravy',
 'recipes/berry-flat-croissant-sandwich',
 'recipes/chinese-sausage-baked-rice',
 'recipes/jalapeno-bite',
 'recipes/garlic-and-pepper-flavored-rice-noodles-with-shredded-chicken',
 'อกไก่',
 'recipes/thai-barbecue-gimbap',
 'recipes/apple-cider-latte',
 'recipes/japanese-salmon-fried-rice',
 'recipes/flower-shaped-kanom-sai-sai',
 '/food-tips']

In [5]:
url_pattern = re.compile(r'^(recipes/|/food-tips)')

# Filter out only URLs
urls = [item for item in links if url_pattern.match(item)]

print(urls)

['recipes/yoghurt-cake', 'recipes/rice-with-chicken-gravy', 'recipes/berry-flat-croissant-sandwich', 'recipes/chinese-sausage-baked-rice', 'recipes/jalapeno-bite', 'recipes/garlic-and-pepper-flavored-rice-noodles-with-shredded-chicken', 'recipes/thai-barbecue-gimbap', 'recipes/apple-cider-latte', 'recipes/japanese-salmon-fried-rice', 'recipes/flower-shaped-kanom-sai-sai', '/food-tips']


In [6]:
urls.pop(-1)

'/food-tips'

In [7]:
urls

['recipes/yoghurt-cake',
 'recipes/rice-with-chicken-gravy',
 'recipes/berry-flat-croissant-sandwich',
 'recipes/chinese-sausage-baked-rice',
 'recipes/jalapeno-bite',
 'recipes/garlic-and-pepper-flavored-rice-noodles-with-shredded-chicken',
 'recipes/thai-barbecue-gimbap',
 'recipes/apple-cider-latte',
 'recipes/japanese-salmon-fried-rice',
 'recipes/flower-shaped-kanom-sai-sai']

In [8]:
# download a web page
url = 'https://www.wongnai.com/recipes/rice-with-chicken-gravy'
downloaded = fetch_url(url)
downloaded is None  # assuming the download was successful


# extract information from HTML
result = extract(downloaded)
print(result)

มาโชว์ฝีมือระดับเหลากันกับ “ข้าวหน้าไก่ราชวงศ์” เมนูสุดคลาสสิกฉบับร้านดังในตำนาน ราดน้ำฉ่ำ ๆ พร้อมเนื้อไก่นุ่ม ๆ รสเข้มข้นเข้าเนื้อ เสิร์ฟพร้อมข้าวสวยร้อน ๆ แค่นี้ก็น้ำลายสอแล้วค่ะ โดยวันนี้เราจะใช้ เอโร่ ข้าวหอมปทุมธานี ที่มีความนุ่ม หอม หุงง่าย สวยเป็นเมล็ด แถมราคาประหยัดอีกต่างหาก ใครจะเอาสูตรข้าวหน้าไก่ราชวงศ์แบบนี้ไปทำขายก็ขอแนะนำเลยค่ะ ถ้าเพื่อน ๆ อยากรู้วิธีทำกันแล้วก็มาเข้าครัวกันเลย!
วัตถุดิบ
เอโร่ ข้าวหอมปทุมธานี 200 กรัม
-
ไก่ 1 กิโลกรัม
-
ซีอิ๊วขาว (สำหรับน้ำราด) 2 ช้อนโต๊ะ
-
ซีอิ๊วขาว (สำหรับหมัก) 1 ช้อนโต๊ะ
-
ซีอิ๊วดำหวาน 3 ช้อนโต๊ะ
-
พริกไทยป่น 1 ช้อนชา
-
แป้งข้าวโพด (สำหรับหมัก) 2 ช้อนโต๊ะ
-
แป้งข้าวโพด 7 ช้อนโต๊ะ
-
น้ำมันงา 2 ช้อนโต๊ะ
-
น้ำมันพืช 50 มิลลิลิตร
-
น้ำสต๊อก 1,600 มิลลิลิตร
-
ซอสปรุงรส 1 ช้อนโต๊ะ
-
น้ำตาลทราย 3 ช้อนโต๊ะ
-
พริกชี้ฟ้าเขียว สำหรับเสิร์ฟเคียง
-
ผักชี สำหรับเสิร์ฟเคียง
-
วิธีทำ
STEP 1 : หมักไก่และหุงข้าว
หั่นอกไก่เป็นชิ้นขนาดพอดีคำ จากนั้นนำไปหมักกับซีอิ๊วขาว ซีอิ๊วดำหวาน พริกไทยป่น และแป้งข้าวโพด พักไว้ 15 นาที
-
ซาว เอโร่ ข้าวหอมปทุมธานี ให้สะ

In [9]:
by_line = result.split('\n')
by_line = [line for line in by_line if line.strip() != '-']

In [10]:
re.findall(r'“(.*?)”', by_line[0])

['ข้าวหน้าไก่ราชวงศ์']

In [11]:
def get_wongnai_recipe(by_line):
    recipes = []
    collecting_ingredients = False

    for line in by_line:
        if line == 'วัตถุดิบ':
            collecting_ingredients = True
        elif line == 'วิธีทำ':
            break
        elif collecting_ingredients:
            recipes.append(line)

    return recipes

In [12]:
def get_wongnai_method(by_line):
    recipes = []
    collecting_ingredients = False

    for line in by_line:
        if line == 'วิธีทำ':
            collecting_ingredients = True
        elif collecting_ingredients:
            recipes.append(line)

    return recipes

In [13]:
recipes = get_wongnai_recipe(by_line)

In [14]:
recipes

['เอโร่ ข้าวหอมปทุมธานี 200 กรัม',
 'ไก่ 1 กิโลกรัม',
 'ซีอิ๊วขาว (สำหรับน้ำราด) 2 ช้อนโต๊ะ',
 'ซีอิ๊วขาว (สำหรับหมัก) 1 ช้อนโต๊ะ',
 'ซีอิ๊วดำหวาน 3 ช้อนโต๊ะ',
 'พริกไทยป่น 1 ช้อนชา',
 'แป้งข้าวโพด (สำหรับหมัก) 2 ช้อนโต๊ะ',
 'แป้งข้าวโพด 7 ช้อนโต๊ะ',
 'น้ำมันงา 2 ช้อนโต๊ะ',
 'น้ำมันพืช 50 มิลลิลิตร',
 'น้ำสต๊อก 1,600 มิลลิลิตร',
 'ซอสปรุงรส 1 ช้อนโต๊ะ',
 'น้ำตาลทราย 3 ช้อนโต๊ะ',
 'พริกชี้ฟ้าเขียว สำหรับเสิร์ฟเคียง',
 'ผักชี สำหรับเสิร์ฟเคียง']

In [15]:
method = get_wongnai_method(by_line)

In [16]:
method

['STEP 1 : หมักไก่และหุงข้าว',
 'หั่นอกไก่เป็นชิ้นขนาดพอดีคำ จากนั้นนำไปหมักกับซีอิ๊วขาว ซีอิ๊วดำหวาน พริกไทยป่น และแป้งข้าวโพด พักไว้ 15 นาที',
 'ซาว เอโร่ ข้าวหอมปทุมธานี ให้สะอาด จากนั้นใส่น้ำ1.3-1.5 ส่วน ต่อข้าว 1 ส่วน และนำไปหุงให้สุก',
 'STEP 2 : ทำหน้าไก่',
 'ตั้งกระทะใบใหญ่ใส่น้ำมันพืชและน้ำมันงา ใส่ไก่ที่หมักไว้ลงผัดด้วยไฟกลางจนสุกดี ใส่น้ำสต๊อก พอเดือด คอยช้อนฟองออกให้น้ำใส',
 'ปรุงรสด้วยซีอิ๊วขาว ซอสปรุงรส และน้ำตาลทราย เคี่ยวไฟกลางเป็นเวลา 25 นาที',
 'ละลายแป้งข้าวโพดกับน้ำสะอาด ค่อย ๆ เทใส่ลงไป คนเร็ว ๆ ให้แป้งไม่จับตัวเป็นก้อน แล้วเคี่ยวจนน้ำข้นและใสดี',
 'STEP 3 : จัดเสิร์ฟ',
 'ตักเสิร์ฟหน้าไก่กับ เอโร่ ข้าวหอมปทุมธานี ที่หุงไว้ เคียงด้วยพริกชี้ฟ้าเขียวหั่นแว่นและผักชี',
 'วิธีทำ “ข้าวหน้าไก่ราชวงศ์” ไม่ยากเลยใช่ไหมคะ ข้าวสวยร้อน ๆ มาพร้อมน้ำราดหน้าไก่ชุ่มฉ่ำ ตักเข้าปากเต็ม ๆ คำ ได้รสชาติกลมกล่อมถูกใจ กินเพลินจนคำสุดท้ายเลยละค่ะ เอโร่ ข้าวหอมปทุมธานี ก็หอมนุ่มกำลังดี ได้รสสัมผัสที่ดีกว่าข้าวขาวทั่ว ๆ ไป เหมาะกับทำเป็นข้าวสวยแบบนี้มาก ๆ ทั้งหอม นุ่ม ทั้งคุ้มค่า กินกับเมนู

In [17]:
class Agent:
    def __init__(self, model_con : str, api_key :str, menu : list[str],recipes : list[str], method : list[str]):
        self.api_key = api_key
        self.menu = menu
        self.recipes = recipes
        self.method = method
        self.model_con = model_con
    def select_model(self):
        if self.model_con == "typhoon":
            print("Using Typhoon model")
            self.client = OpenAI(api_key=self.api_key, base_url="https://api.opentyphoon.ai/v1")
        else:
            print("No model selected")
    def get_response(self, user_prompt : str, model = "typhoon-instruct", max_tokens = 1000, temperature = 1.0, top_p = 0.1):
        self.select_model()
        messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": user_prompt},
    ]
        response = self.client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
        )
        content = response.choices[0].message.content
        return content
    
    def gen_question(self):
        user_prompt = f"""
        You are a recipe assessment expert. Your task is to create insightful multiple-choice questions (MCQs) in Thai that assess a user's comprehension of a given recipe and the cooking techniques involved.

        Input:

        * Recipe Name: {self.menu}
        * Ingredients: {self.recipes}
        * Instructions/Method: {self.method}

        Output: 

        Generate 3-5 MCQs that cover these areas:

        1. Ingredient Knowledge: Questions about ingredient substitutions, their functions in the dish, or how to select high-quality ingredients.
        2. Technique Understanding: Questions about specific cooking methods used in the recipe, their purpose, or potential challenges/troubleshooting.
        3. Recipe Comprehension: Questions about the overall process, the order of steps, or the reasoning behind certain instructions.
        4. Food Safety: Questions (where applicable) about safe handling practices, temperature checks, or storage instructions.

        Each MCQ should have:

        * A clear and concise question stem.
        * 4 answer choices (a, b, c, d) with only ONE correct answer.
        * A brief explanation of the correct answer, highlighting key points from the recipe or general cooking knowledge.
        
        This is a sample format for the MCQs (dictionary format):
        * "question": "คำในข้อใดต่อไปนี้สะกดถูกต้องทุกคำ", "a": "ผู้เรียน กฎ บันได เที่ยวอนุญาต", "b": "ไล่เลี่ยง ซึมเซ่บ แกงบวช สถิต นกอินทรีย์", "c": "อัตราคัด โล่ห์ ขบถ นกพิราบ เส้นทแยง", "d": "สำปรางแช่ง โขมย อะไหล่ อุโมงก์ โอกาส", "e": "ไข่มุก ผักคะน้า มุกตลก กะเทยะ รสแซ่บ", "answer": "e", "subject": "Thai"
        """
        mcq = self.get_response(user_prompt)
        return mcq
        
        
        
        
        

In [18]:
Typhoon_API_KEY = "sk-eRXr8uXlfzhTn63Jr2txVkVXZrc106WR1Xuzh4xPuSwZKOdg"

In [19]:
import time
from tqdm.notebook import tqdm

In [20]:
mcqs = []
for url in tqdm(urls):
    try:
        print(f"Processing {url}")
        full_url = f"https://www.wongnai.com/{url}"
        downloaded = fetch_url(full_url)
        result = extract(downloaded)
        by_line = result.split('\n')
        by_line = [line for line in by_line if line.strip() != '-']
        menu = re.findall(r'“(.*?)”', by_line[0])
        recipes = get_wongnai_recipe(by_line)
        method = get_wongnai_method(by_line)
        time.sleep(5)
        agent = Agent(model_con = "typhoon", api_key = Typhoon_API_KEY, menu = menu, recipes = recipes, method = method)
        mcq = agent.gen_question()
        mcqs.append(mcq)
    except Exception as e:
        print(f"Error processing {url}")
        print(e)
        continue

  0%|          | 0/10 [00:00<?, ?it/s]

Processing recipes/yoghurt-cake
Using Typhoon model
Processing recipes/rice-with-chicken-gravy
Using Typhoon model
Processing recipes/berry-flat-croissant-sandwich
Using Typhoon model
Processing recipes/chinese-sausage-baked-rice
Using Typhoon model
Processing recipes/jalapeno-bite
Using Typhoon model
Processing recipes/garlic-and-pepper-flavored-rice-noodles-with-shredded-chicken
Using Typhoon model
Processing recipes/thai-barbecue-gimbap
Using Typhoon model
Processing recipes/apple-cider-latte
Using Typhoon model
Processing recipes/japanese-salmon-fried-rice
Using Typhoon model
Processing recipes/flower-shaped-kanom-sai-sai
Using Typhoon model


In [26]:
print(mcqs[0])

1. Ingredient Knowledge:
* คำถาม: ในสูตรโยเกิร์ตเค้กนี้ แป้งมันสำปะหลังต้นสนมีหน้าที่สำคัญอย่างไร?
	+ a) เพิ่มความหวานให้กับเค้ก
	+ b) ช่วยทำให้เค้กนุ่มฟู
	+ c) ทำให้เค้กมีกลิ่นหอมเฉพาะตัว
	+ d) ทำให้น้ำตาลละลายได้ดีขึ้น
* คำตอบ: b) ช่วยทำให้เค้กนุ่มฟู
* สามารถอธิบายได้ว่า แป้งมันสำปะหลังต้นสนเป็นส่วนประกอบที่สำคัญในการทำเค้ก เพราะมันช่วยเพิ่มปริมาณแก๊สที่ทำให้เค้กขยายตัวและทำให้เนื้อเค้กมีความนุ่มฟู

2. Technique Understanding:
* คำถาม: การตีไข่ขาวให้ขึ้นฟองก่อนการผสมกับส่วนผสมอื่นๆ มีประโยชน์อย่างไร?
	+ a) เพื่อให้เค้กมีสีสันที่สวยงาม
	+ b) เพื่อให้เค้กมีรสชาติที่หวานมากขึ้น
	+ c) เพื่อให้เค้กมีเนื้อสัมผัสที่นุ่มฟู
	+ d) เพื่อให้เค้กมีกลิ่นหอมเฉพาะตัว
* คำตอบ: c) เพื่อให้เค้กมีเนื้อสัมผัสที่นุ่มฟู
* สามารถอธิบายได้ว่า การตีไข่ขาวให้ขึ้นฟองเป็นการเพิ่มปริมาณแก๊สในเค้ก ซึ่งจะช่วยให้เค้กมีเนื้อสัมผัสที่นุ่มฟูและไม่ยุ่ย

3. Recipe Comprehension:
* คำถาม: ในการทำโยเกิร์ตเค้ก ควรเริ่มขั้นตอนไหนเป็นคนแรก?
	+ a) แบ่งไข่ขาวและไข่แดง
	+ b) เทส่วนผสมเค้กโยเกิร์ตลงในพิมพ์
	+ c) ตีส่วนผสมโยเกิร์ต

In [33]:
import re

def extract_exam_items(text):
  """Extracts exam items from Thai text and returns them as a list of dictionaries.

  Args:
      text: The text containing the exam items.

  Returns:
      A list of dictionaries, each representing an exam item.
  """

  # Regular expressions for extracting different parts of the exam items
  question_pattern = r"คำถาม:\s*(.+)"
  choice_pattern = r"\+\s*([a-d])\)\s*(.+)"
  answer_pattern = r"คำตอบ:\s*([a-d])\)\s*(.+)"

  # Find all questions in the text
  question_matches = re.finditer(question_pattern, text)

  exam_items = []
  for question_match in question_matches:
    item = {
        "question": question_match.group(1).strip(),
        "a": "",
        "b": "",
        "c": "",
        "d": "",
        "answer": "",
        "subject": "Thai food"
    }

    # Find choices and answer for the current question
    start = question_match.end()
    end = text.find("คำถาม:", start) if text.find("คำถาม:", start) != -1 else len(text)
    question_text = text[start:end]

    for choice_match in re.finditer(choice_pattern, question_text):
      item[choice_match.group(1)] = choice_match.group(2).strip()

    answer_match = re.search(answer_pattern, question_text)
    if answer_match:
      item["answer"] = answer_match.group(1)

    exam_items.append(item)

  return exam_items

In [59]:
import json

In [70]:
import json

def extract_valid_json(items):
    """Extracts valid JSON objects from a list of strings."""
    valid_json_objects = []
    for item in items:
        # Attempt to load the item as JSON
        try:
            loaded_json = json.loads(item)  # Attempt to load as full JSON first
            valid_json_objects.append(loaded_json)
        except json.JSONDecodeError:  #If not, try to create a JSON
            for i in range(len(item)):
                try:
                    loaded_json = json.loads("{" + str(item[i:]) + "}")
                    valid_json_objects.append(loaded_json)
                    break  # Stop if a valid JSON is found
                except json.JSONDecodeError:
                    continue  # Try the next substring if not valid
    return valid_json_objects


items = extract_valid_json(mcqs[1].split("*")) 


In [69]:
print(items)

[{'คำถาม': 'ส่วนผสมใดใช้ในการหมักไก่?', 'a': 'ซีอิ๊วขาว, ซีอิ๊วดำหวาน, พริกไทยป่น, แป้งข้าวโพด', 'b': 'น้ำมันงา, น้ำมันพืช, น้ำสต๊อก, ซอสปรุงรส', 'c': 'น้ำตาลทราย, พริกชี้ฟ้าเขียว, ผักชี, เอโร่ ข้าวหอมปทุมธานี', 'd': 'ไข่ไก่, กระเทียม, มิริน, หอมหัวใหญ่', 'answer': 'a', 'subject': 'Ingredient Knowledge'}, {'คำถาม': 'การทอดไก่ควรใช้ความร้อนระดับใด?', 'a': 'ต่ำ', 'b': 'ปานกลาง', 'c': 'สูง', 'd': 'สุดยอด', 'answer': 'b', 'subject': 'Technique Understanding'}, {'คำถาม': 'ขั้นตอนการทำข้าวหน้าไก่ราชวงศ์คืออะไรบ้าง?', 'a': 'หมักไก่, หุงข้าว, ทำหน้าไก่, จัดเสิร์ฟ', 'b': 'ทำหน้าไก่, หุงข้าว, หมักไก่, จัดเสิร์ฟ', 'c': 'จัดเสิร์ฟ, หุงข้าว, ทำหน้าไก่, หมักไก่', 'd': 'หมักไก่, ทำหน้าไก่, หุงข้าว, จัดเสิร์ฟ', 'answer': 'a', 'subject': 'Recipe Comprehension'}, {'คำถาม': 'การละลายแป้งข้าวโพดต้องทำอย่างไร?', 'a': 'ผสมแป้งข้าวโพดกับน้ำแล้วคนให้เข้ากัน', 'b': 'เติมแป้งข้าวโพดลงไปในน้ำสต๊อก', 'c': 'ผสมแป้งข้าวโพดกับน้ำสต๊อกแล้วคนให้เข้ากัน', 'd': 'ผสมแป้งข้าวโพดกับน้ำมันงา', 'answer': 'c', 'subject': 'Tec

In [91]:
print(mcqs[6])

1. Ingredient Knowledge:
* คำถาม: คุณสามารถใช้ข้าวชนิดอื่นแทนข้าวญี่ปุ่นในการทำคิมบับหรือไม่?
	+ a. ใช้ข้าวกล้อง
	+ b. ใช้ข้าวหอมมะลิ
	+ c. ใช้ข้าวไรย์
	+ d. ใช้ข้าวสังเกตี
	+ answer: b
	+ subject: Thai
	+ explanation: ในสูตรนี้แนะนำให้ใช้ข้าวญี่ปุ่น แต่คุณสามารถใช้ข้าวชนิดอื่นๆ ได้ เช่น ข้าวหอมมะลิ ซึ่งจะมีรสชาติและสัมผัสที่แตกต่างกัน
2. Technique Understanding:
* คำถาม: สามารถย่างหมูในขั้นตอนการทำให้สุกหรือไม่?
	+ a. ไม่สามารถย่างได้ ต้องหุงเท่านั้น
	+ b. ควรย่างให้สุกมากเพื่อป้องกันเชื้อโรค
	+ c. ควรย่างให้พอดี ไม่ควรสุกเกินไป
	+ d. ควรย่างให้สุกสนิทเพื่อให้หมูนุ่ม
	+ answer: c
	+ subject: Thai
	+ explanation: สูตรแนะนำให้ย่างหมูให้พอดี ไม่ควรสุกเกินไป เพื่อป้องกันหมูเหนียวและแข็ง
3. Recipe Comprehension:
* คำถาม: ขั้นตอนการห่อคิมบับต้องทำอย่างไร?
	+ a. วางข้าวบนแผ่นสาหร่าย แล้ววางเนื้อหมูลงไป
	+ b. วางเนื้อหมูบนแผ่นสาหร่าย แล้วยังไม่ได้วางข้าว
	+ c. วางผักลงบนแผ่นสาหร่าย ก่อนที่จะวางข้าวและเนื้อหมู
	+ d. วางข้าวบนแผ่นสาหร่าย แล้ววางผักลงไป
	+ answer: a
	+ subject: Thai
	+ explanati

In [95]:
import re
import json

def extract_exam_items(texts):
    """Extracts exam items from a list of Thai texts in various formats and returns them as a list of dictionaries.

    Args:
        texts: A list of strings, each containing text with exam items.

    Returns:
        A list of dictionaries, each representing an exam item.
    """
    all_exam_items = []
    for text in texts:
        # Check if the text is in JSON format
        if text.startswith("*"):
            try:
                items = extract_valid_json(text.split("*"))
                for item in items:
                    # Extract the question, answer, and subject values
                    question = item.pop("คำถาม")
                    answer = item.pop("answer")
                    subject = item.pop("subject")
                    
                    # Create a new dictionary with the desired order
                    new_item = {
                        "question": question,
                        "a": item.pop("a"),
                        "b": item.pop("b"),
                        "c": item.pop("c"),
                        "d": item.pop("d"),
                        "answer": answer,
                        "subject": "Food"
                    }

                    # Update the original item with the new dictionary
                    item.clear()  # Clear the original dictionary
                    item.update(new_item)  # Update with the new dictionary
                all_exam_items.extend(items)
                continue  # Skip the rest of the loop if JSON parsing is successful
            except json.JSONDecodeError:
                pass  # If not valid JSON, continue to the next format

        # Regular expressions for extracting different parts of the exam items
        question_pattern = r"คำถาม:\s*(.+)"
        choice_pattern = r"([a-e])[.)]\s*(.+)"
        answer_pattern = r"(?:คำตอบ|answer):\s*([a-d])[\)]*" 

        # Find all questions in the current text
        question_matches = re.finditer(question_pattern, text)

        exam_items = []
        for question_match in question_matches:
            item = {
                "question": question_match.group(1).strip(),
                "a": "",
                "b": "",
                "c": "",
                "d": "",
                "answer": "",
                "subject": "Thai food"  # Default subject (can be updated from text)
            }

            # Find choices and answer for the current question
            start = question_match.end()
            end = text.find("คำถาม:", start) if text.find("คำถาม:", start) != -1 else len(text)
            question_text = text[start:end]

            for choice_match in re.finditer(choice_pattern, question_text):
                item[choice_match.group(1)] = choice_match.group(2).strip()

            answer_match = re.search(answer_pattern, question_text)
            if answer_match:
                item["answer"] = answer_match.group(1)

            # # Update subject if found in the question text
            # subject_match = re.search(r"subject:\s*(\w+)", question_text)
            # if subject_match:
            #     item["subject"] = subject_match.group(1)

            exam_items.append(item)
        all_exam_items.extend(exam_items)
    return all_exam_items

# Extract exam items from the list of texts
extracted_items = extract_exam_items(mcqs)
for item in extracted_items:
  print(item)


{'question': 'ในสูตรโยเกิร์ตเค้กนี้ แป้งมันสำปะหลังต้นสนมีหน้าที่สำคัญอย่างไร?', 'a': 'เพิ่มความหวานให้กับเค้ก', 'b': 'ช่วยทำให้เค้กนุ่มฟู', 'c': 'ทำให้เค้กมีกลิ่นหอมเฉพาะตัว', 'd': 'ทำให้น้ำตาลละลายได้ดีขึ้น', 'answer': 'b', 'subject': 'Thai food'}
{'question': 'การตีไข่ขาวให้ขึ้นฟองก่อนการผสมกับส่วนผสมอื่นๆ มีประโยชน์อย่างไร?', 'a': 'เพื่อให้เค้กมีสีสันที่สวยงาม', 'b': 'เพื่อให้เค้กมีรสชาติที่หวานมากขึ้น', 'c': 'เพื่อให้เค้กมีเนื้อสัมผัสที่นุ่มฟู', 'd': 'เพื่อให้เค้กมีกลิ่นหอมเฉพาะตัว', 'answer': 'c', 'subject': 'Thai food'}
{'question': 'ในการทำโยเกิร์ตเค้ก ควรเริ่มขั้นตอนไหนเป็นคนแรก?', 'a': 'แบ่งไข่ขาวและไข่แดง', 'b': 'เทส่วนผสมเค้กโยเกิร์ตลงในพิมพ์', 'c': 'ตีส่วนผสมโยเกิร์ตเค้ก', 'd': 'วางพิมพ์บนถาดอบก้นลึก', 'answer': 'c', 'subject': 'Thai food'}
{'question': 'หลังจากอบโยเกิร์ตเค้กเสร็จ ควรรอให้เค้กเย็นลงก่อนหรือไม่?', 'a': 'ไม่จำเป็นต้องรอ ให้รับประทานได้เลย', 'b': 'ควรรอให้เค้กอุ่นๆ ก่อนจึงค่อยรับประทาน', 'c': 'ควรรอให้เค้กเย็นลงก่อนจึงค่อยรับประทาน', 'd': 'ควรรอให้เค้กเย็นลงก

In [75]:
import jsonlines

In [96]:
with jsonlines.open("exam_gen_data.jsonl", mode='w') as writer:
    for item in extracted_items:  # exam_data คือ list ของ dictionaries ที่เก็บข้อมูลข้อสอบ
        writer.write(item)

In [22]:
with open("MCQs_saved.txt", "w") as f:
    for mcq in mcqs:
        f.write(mcq)
        f.write("\n")